In [1]:
import re

In [2]:
def remove_brackets(source, id):
    reg = '\(([^\(]*?)\)'
    m = re.search(reg, source)
    if m is None:
        return None, None
    new_source = re.sub(reg, str(id), source, count=1)
    return new_source, m.group(1)

In [3]:
class logic_base:
    def __init__(self, input):
        self.my_stack = []
        self.source = input
        final = input
        while 1:
            input, tmp = remove_brackets(input, len(self.my_stack))
            if input is None:
                break
            final = input
            self.my_stack.append(tmp)
        self.my_stack.append(final)
    def get_result(self):
        root = self.my_stack[-1]
        m = re.match('\s*([0-9]+)\s*$',root)
        if m is not None:
            root = self.my_stack[int(m.group(1))]
        reg = '(\d+)'
        while 1:
            m = re.search(reg, root)
            if m is None:
                break
            new = '(' + self.my_stack[int(m.group(1))] + ')'
            root = re.sub(reg, new, root, count=1)
        return root
    def merge_items(self, logic):
        reg0 = '(\d+)'
        reg1 = 'neg\s+(\d+)'
        flag = False
        for i in range(len(self.my_stack)):
            target = self.my_stack[i]
            if logic not in target:
                continue
            m = re.search(reg1, target)
            if m is not None:
                continue
            m = re.search(reg0, target)
            if m is None:
                continue
            for j in re.findall(reg0,target):
                child = self.my_stack[int(j)]
                if logic not in child:
                    continue
                new_reg = "(^|\s)" + j + "(\s|$)"
                self.my_stack[i] = re.sub(new_reg, ' '+child+' ', self.my_stack[i], count=1)
                self.my_stack[i] = self.my_stack[i].strip()
                flag = True
        if flag:
            self.merge_items(logic)

In [4]:
class ordering(logic_base):
    def run(self):
        flag = False
        for i in range(len(self.my_stack)):
            new_source = self.add_brackets(self.my_stack[i])
            if self.my_stack[i] != new_source:
                self.my_stack[i] = new_source
                flag = True
        return flag
    def add_brackets(self, source):
        reg = "\s+(and|or|imp|iff)\s+"
        if len(re.findall(reg,source)) < 2:
            return source
        reg_and = "(neg\s+)?\S+\s+and\s+(neg\s+)?\S+"
        m = re.search(reg_and, source)
        if m is not None:
            return re.sub(reg_and, "("+m.group(0)+")", source, count=1)
        reg_or = "(neg\s+)?\S+\s+or\s+(neg\s+)?\S+"
        m = re.search(reg_or, source)
        if m is not None:
            return re.sub(reg_or, "("+m.group(0)+")", source, count=1)
        reg_imp = "(neg\s+)?\S+\s+imp\s+(neg\s+)?\S+"
        m = re.search(reg_imp, source)
        if m is not None:
            return re.sub(reg_imp, "("+m.group(0)+")", source, count=1)
        reg_iff = "(neg\s+)?\S+\s+iff\s+(neg\s+)?\S+"
        m = re.search(reg_iff, source)
        if m is not None:
            return re.sub(reg_iff, "("+m.group(0)+")", source, count=1)


In [5]:
class replace_iff(logic_base):
    def run(self):
        final = len(self.my_stack) - 1
        flag = self.replace_all_iff()
        self.my_stack.append(self.my_stack[final])
        return flag
    def replace_all_iff(self):
        flag = False
        for i in range(len(self.my_stack)):
            ans = self.replace_iff_inner(self.my_stack[i], len(self.my_stack))
            if ans is None:
                continue
            self.my_stack[i] = ans[0]
            self.my_stack.append(ans[1])
            self.my_stack.append(ans[2])
            flag = True
        return flag
    def replace_iff_inner(self,source,id):
        reg = '^(.*?)\s+iff\s+(.*?)$'
        m = re.search(reg, source)
        if m is None:
            return None
        a, b = m.group(1), m.group(2)
        return (str(id) + ' and ' + str(id+1), a + ' imp ' + b, b + ' imp ' + a)


In [6]:
class replace_imp(logic_base):
    def run(self):
        flag = False
        for i in range(len(self.my_stack)):
            ans = self.replace_imp_inner(self.my_stack[i])
            if ans is None:
                continue
            self.my_stack[i] = ans
            flag = True
        return flag
    def replace_imp_inner(self,source):
        reg = '^(.*?)\s+imp\s+(.*?)$'
        m = re.search(reg, source)
        if m is None:
            return None
        a, b = m.group(1), m.group(2)
        if 'neg ' in a:
            return a.replace('neg ','') + ' or ' + b
        return 'neg ' + a + ' or ' + b


In [7]:
class de_morgan(logic_base):
    def run(self):
        reg = 'neg\s+(\d+)'
        flag = False
        final = len(self.my_stack) - 1
        for i in range(len(self.my_stack)):
            target = self.my_stack[i]
            m = re.search(reg, target)
            if m is None:
                continue
            flag = True
            child = self.my_stack[int(m.group(1))]
            self.my_stack[i] = re.sub(reg, str(len(self.my_stack)), target, count=1)
            self.my_stack.append(self.doing_de_morgan(child))
            break
        self.my_stack.append(self.my_stack[final])
        return flag
    def doing_de_morgan(self,source):
        items = re.split('\s+',source)
        new_items = []
        for item in items:
            if item == 'or':
                new_items.append('and')
            elif item == 'and':
                new_items.append('or')
            elif item == 'neg':
                new_items.append('neg')
            elif len(item.strip()) > 0:
                new_items.append('neg')
                new_items.append(item)
        for i in range(len(new_items)-1):
            if new_items[i] == 'neg':
                if new_items[i+1] == 'neg':
                    new_items[i] = ''
                    new_items[i+1] = ''
        return ' '.join([i for i in new_items if len(i)>0])


In [8]:
class distributive(logic_base):
    def run(self):
        flag = False
        reg = '(\d+)'
        final = len(self.my_stack) - 1
        for i in range(len(self.my_stack)):
            target = self.my_stack[i]
            if 'or' not in self.my_stack[i]:
                continue
            m = re.search(reg, target)
            if m is None:
                continue
            for j in re.findall(reg,target):
                child = self.my_stack[int(j)]
                if 'and' not in child:
                    continue
                new_reg = "(^|\s)" + j + "(\s|$)"
                items = re.split('\s+and\s+',child)
                tmp_list = [str(j) for j in range(len(self.my_stack),len(self.my_stack)+len(items))]
                for item in items:
                    self.my_stack.append(re.sub(new_reg, ' '+item+' ', target).strip())
                self.my_stack[i] = ' and '.join(tmp_list)
                flag = True
            if flag:
                break
        self.my_stack.append(self.my_stack[final])
        return flag


In [9]:
class simplification(logic_base):
    def run(self):
        old = self.get_result()
        for i in range(len(self.my_stack)):
            self.my_stack[i] = self.reducing_or(self.my_stack[i])
            #self.my_stack[i] = self.reducing_and(self.my_stack[i])
        final = self.my_stack[-1]
        self.my_stack[-1] = self.reducing_and(final)
        return len(old) != len(self.get_result())
    def reducing_and(self,target):
        if 'and' not in target:
            return target
        items = set(re.split('\s+and\s+', target))
        for item in list(items):
            if ('neg '+ item) in items:
                return ''
            if re.match('\d+$',item) is None:
                continue
            value = self.my_stack[int(item)]
            if self.my_stack.count(value) > 1:
                value = ''
                self.my_stack[int(item)] = ''
            if value == '':
                items.remove(item)
        return ' and '.join(list(items))
    def reducing_or(self,target):
        if 'or' not in target:
            return target
        items = set(re.split('\s+or\s+', target))
        for item in list(items):
            if ('neg '+item) in items:
                return ''
        return ' or '.join(list(items))

In [10]:
def merging(source):
    old = source.get_result()
    source.merge_items('or')
    source.merge_items('and')
    return old != source.get_result()

In [11]:
def run(input):
    all_strings = []
    #all_strings.append(input)
    zero = ordering(input)
    while zero.run():
        zero = ordering(zero.get_result())
    merging(zero)

    one = replace_iff(zero.get_result())
    one.run()
    all_strings.append(one.get_result())
    merging(one)
        
    two = replace_imp(one.get_result())
    two.run()
    all_strings.append(two.get_result())
    merging(two)

    three, four = None, None
    old = two.get_result()
    three = de_morgan(old)
    while three.run():
        pass
    all_strings.append(three.get_result())
    merging(three)
    three_helf = simplification(three.get_result())
    three_helf.run()

    four = distributive(three_helf.get_result())
    while four.run():
        pass
    merging(four)
    five = simplification(four.get_result())
    five.run()
    all_strings.append(five.get_result())
    return all_strings

In [12]:
def process(input):    
    a = input.replace(">", " imp ", 10).replace("!", " neg ", 10).replace("&", " and ", 10).replace("|", " or ", 10)
    b = run(str(a))
    c = b[len(b) - 1]
    #d = c.replace("imp", ">", 10).replace("neg", "!", 10).replace("and", "&", 10).replace("or", "|", 10).replace(" ", "", 20) 
    d = []
    for i in b:
        d.append(i.replace("imp", ">", 10).replace("neg", "!", 10).replace("and", "&", 10).replace("or", "|", 10).replace(" ", "", 20) )
    return d 

In [52]:
def get_clause(data):
    
    clauses = []
    all_clauses = data.split("&")
    for c in all_clauses:
        clauses.append(c)
    
    return clauses

In [123]:
def process_input_data(data):
    
    _cfns = []
    final_clauses = []
    
    for d in data:
        _cfns.append(process(d)[-1])
    
    for cfn in _cfns:
        invidual_cfns = get_clause(cfn)
        
        final_clauses += invidual_cfns
        
    return final_clauses 

## Program Starts here

In [125]:
# ### Raw Data from STDIN/File - FOR Hacker Rank
# data = []
# for line in fileinput.input():
#     data.append(line)
# data = [ d.rstrip().split()  for d in data]
 
### Raw Data from STDIN/File  - FOR LAPTOP 
with open("input.txt") as f:
    data = f.readlines()
data = [ d.rstrip()  for d in data]

input_data = data[:-1]
conclusion = data[-1]

conclusion_with_negation = [f"!({conclusion})"]
print(conclusion_with_negation)

all_clauses = process_input_data(input_data)
print(all_clauses)
all_clauses += process_input_data(conclusion_with_negation)

print(all_clauses)

['!(R>!H)']
['!S|E|!W', '!W|!P|H', '!H|!R', '!R|G', 'W', 'R', 'S']
['!S|E|!W', '!W|!P|H', '!H|!R', '!R|G', 'W', 'R', 'S', 'R', 'H']


In [141]:

def to_cnf(s):
    """
    [Page 253]
    Convert a propositional logical sentence to conjunctive normal form.
    That is, to the form ((A | ~B | ...) & (B | C | ...) & ...)
    >>> to_cnf('~(B | C)')
    (~B & ~C)
    """
    s = expr(s)
    if isinstance(s, str):
        s = expr(s)
    s = eliminate_implications(s)  # Steps 1, 2 from p. 253
    s = move_not_inwards(s)  # Step 3
    return distribute_and_over_or(s)  # Step 4


def eliminate_implications(s):
    """Change implications into equivalent form with only &, |, and ~ as logical operators."""
    s = expr(s)
    if not s.args or is_symbol(s.op):
        return s  # Atoms are unchanged.
    args = list(map(eliminate_implications, s.args))
    a, b = args[0], args[-1]
    if s.op == '==>':
        return b | ~a
    elif s.op == '<==':
        return a | ~b
    elif s.op == '<=>':
        return (a | ~b) & (b | ~a)
    elif s.op == '^':
        assert len(args) == 2  # TODO: relax this restriction
        return (a & ~b) | (~a & b)
    else:
        assert s.op in ('&', '|', '~')
        return Expr(s.op, *args)


def move_not_inwards(s):
    """Rewrite sentence s by moving negation sign inward.
    >>> move_not_inwards(~(A | B))
    (~A & ~B)
    """
    s = expr(s)
    if s.op == '~':
        def NOT(b):
            return move_not_inwards(~b)

        a = s.args[0]
        if a.op == '~':
            return move_not_inwards(a.args[0])  # ~~A ==> A
        if a.op == '&':
            return associate('|', list(map(NOT, a.args)))
        if a.op == '|':
            return associate('&', list(map(NOT, a.args)))
        return s
    elif is_symbol(s.op) or not s.args:
        return s
    else:
        return Expr(s.op, *list(map(move_not_inwards, s.args)))


def distribute_and_over_or(s):
    """Given a sentence s consisting of conjunctions and disjunctions
    of literals, return an equivalent sentence in CNF.
    >>> distribute_and_over_or((A & B) | C)
    ((A | C) & (B | C))
    """
    s = expr(s)
    if s.op == '|':
        s = associate('|', s.args)
        if s.op != '|':
            return distribute_and_over_or(s)
        if len(s.args) == 0:
            return False
        if len(s.args) == 1:
            return distribute_and_over_or(s.args[0])
        conj = first(arg for arg in s.args if arg.op == '&')
        if not conj:
            return s
        others = [a for a in s.args if a is not conj]
        rest = associate('|', others)
        return associate('&', [distribute_and_over_or(c | rest)
                               for c in conj.args])
    elif s.op == '&':
        return associate('&', list(map(distribute_and_over_or, s.args)))
    else:
        return s


def associate(op, args):
    """Given an associative op, return an expression with the same
    meaning as Expr(op, *args), but flattened -- that is, with nested
    instances of the same op promoted to the top level.
    >>> associate('&', [(A&B),(B|C),(B&C)])
    (A & B & (B | C) & B & C)
    >>> associate('|', [A|(B|(C|(A&B)))])
    (A | B | C | (A & B))
    """
    args = dissociate(op, args)
    if len(args) == 0:
        return _op_identity[op]
    elif len(args) == 1:
        return args[0]
    else:
        return Expr(op, *args)


_op_identity = {'&': True, '|': False, '+': 0, '*': 1}


def dissociate(op, args):
    """Given an associative op, return a flattened list result such
    that Expr(op, *result) means the same as Expr(op, *args).
    >>> dissociate('&', [A & B])
    [A, B]
    """
    result = []

    def collect(subargs):
        print(subargs)
        for arg in subargs:
            if arg.op == op:
                collect(arg.args)
            else:
                result.append(arg)

    collect(args)
    return result


def conjuncts(s):
    """Return a list of the conjuncts in the sentence s.
    >>> conjuncts(A & B)
    [A, B]
    >>> conjuncts(A | B)
    [(A | B)]
    """
    return dissociate('&', [s])


def disjuncts(s):
    """Return a list of the disjuncts in the sentence s.
    >>> disjuncts(A | B)
    [A, B]
    >>> disjuncts(A & B)
    [(A & B)]
    """
    return dissociate('|', [s])


In [142]:
def pl_resolve(ci, cj):
    """Return all clauses that can be obtained by resolving clauses ci and cj."""
    clauses = []
    for di in disjuncts(ci):
        for dj in disjuncts(cj):
            if di == ~dj or ~di == dj:
                clauses.append(associate('|', unique(remove_all(di, disjuncts(ci)) + remove_all(dj, disjuncts(cj)))))
    return clauses

In [143]:
def pl_resolution(clauses):
    "Propositional-logic resolution: say if alpha follows from KB. [Fig. 7.12]"
    #clauses = KB.clauses + conjuncts(to_cnf(~alpha))
    new = set()
    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j])
                 for i in range(n) for j in range(i+1, n)]
        for (ci, cj) in pairs:
            resolvents = pl_resolve(ci, cj)
            if FALSE in resolvents: return True
            new = new.union(set(resolvents))
        if new.issubset(set(clauses)): return False
        for c in new:
            if c not in clauses: clauses.append(c)

In [144]:
# ### Raw Data from STDIN/File - FOR Hacker Rank
# data = []
# for line in fileinput.input():
#     data.append(line)
# data = [ d.rstrip().split()  for d in data]
 
### Raw Data from STDIN/File  - FOR LAPTOP 
with open("input.txt") as f:
    data = f.readlines()
data = [ d.rstrip()  for d in data]

input_data = data[:-1]
conclusion = data[-1]

conclusion_with_negation = [f"!({conclusion})"]
print(conclusion_with_negation)

all_clauses = process_input_data(input_data)
print(all_clauses)
all_clauses += process_input_data(conclusion_with_negation)

print(all_clauses)

['!(R>!H)']
['!S|E|!W', '!W|!P|H', '!H|!R', '!R|G', 'W', 'R', 'S']
['!S|E|!W', '!W|!P|H', '!H|!R', '!R|G', 'W', 'R', 'S', 'R', 'H']


In [145]:
pl_resolution(all_clauses)

['!S|E|!W']


AttributeError: 'str' object has no attribute 'op'